<a href="https://colab.research.google.com/github/ABeleris/Forensic/blob/main/Distilbert_Forensic_Analysis_May.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Environment Setup and Package Installation**

**Purpose:** Installs the necessary libraries:

**datasets:** Hugging Face’s library for handling and processing datasets.

**transformers\[torch]:** Provides pre-trained transformer models  that use PyTorch.

**scikit-learn:** Used for data splitting, label encoding, and evaluation metrics.

In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **2. Importing Libraries and Disabling Unwanted Logging**

* The code then imports Python libraries for regular expressions, Torch, Pandas, and others.
* It also disables wandb (Weights & Biases) logging by setting an environment variable

In [ ]:
import re
import torch
import pandas as pd
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
import os
# Disable wandb logging
os.environ["WANDB\_DISABLED"] = "true"

# **3. Preprocessing the Raw Logs**

**a. Loading the Log Templates**

* The file "HDFS.log\_templates.csv" is loaded into a DataFrame.

* A dictionary (log\_templates\_dict) is built that maps each template’s text (with wildcards) to its corresponding EventId.

In [ ]:
raw_log_file = "/content/drive/MyDrive/DISTIL_FILES/input/HDFS.log"  # Update with actual raw log file
log_templates_file = "/content/drive/MyDrive/DISTIL_FILES/input/HDFS.log_templates.csv"
preprocessed_output = "χχχχχ"


In [ ]:
# Load Log Templates
log_templates = pd.read_csv(log_templates_file)
log_templates_dict = {row['EventTemplate']: row['EventId'] for _, row in log_templates.iterrows()}

In [ ]:
log_templates_dict

**b. Defining Helper Functions**

Two functions are defined:

1. extract\_block\_id: Uses a regular expression to extract the block ID (e.g. "blk\_-1608999687919862906") from a log line.

2. match\_log\_to\_template: Iterates over the log templates. For each template, it:

* Converts the template to a regular expression by escaping it and replacing \* with a wildcard pattern (.\*).
* Checks if the log line matches that pattern.
* Returns the corresponding event ID if a match is found.

In [ ]:
# Function to extract BlockId from a log line
def extract_block_id(log_line):
match = re.search(r'blk_-?\d+', log_line)
return match.group(0) if match else None

In [ ]:
# Function to match log line to a template
def match_log_to_template(log_line):
for template, event_id in log_templates_dict.items():

# Convert template to regex (replace [*] with wildcards)
pattern = re.escape(template).replace(r'\[\*\]', '.*')
if re.match(pattern, log_line):
return event_id
return None  # If no match found

**c. Processing Each Raw Log Line**

* The raw log file is opened and processed line by line.
* For each line, the block ID is extracted and the log is matched to a template to get an event ID.
* If both are found, a tuple (block\_id, event\_id, raw log line) is appended to a list.
* Finally, the structured information is saved as a CSV file.
* Outcome: Preprocessed logs are now saved, where each row represents a log line with its associated block and event ID.

In [ ]:
# Process raw logs

preprocessed_data = []
with open(raw_log_file, "r", encoding="utf-8") as file:
for line in file:
line = line.strip()
block_id = extract_block_id(line)
event_id = match_log_to_template(line)

    if block_id and event_id:
        preprocessed_data.append((block_id, event_id, line))


# **4. Creating Event Traces and Grouping Logs**

**a. Loading Preprocessed Logs and Anomaly Labels**

* The preprocessed logs CSV (from Google Drive) is loaded.
* Anomaly labels (which indicate whether a block is “Normal” or “Anomaly”) are loaded and stored as a dictionary mapping from BlockId to Label.

In [ ]:
# Save structured logs
df = pd.DataFrame(preprocessed_data, columns=["BlockId", "EventId", "RawLog"])
df.to_csv(preprocessed_output, index=False)

print(f"Step 1 Complete: Preprocessed logs saved to {preprocessed_output}")

NameError: name 'preprocessed_data' is not defined

In [ ]:
# File Paths

preprocessed_logs_file = "/content/drive/MyDrive/DISTIL_FILES/input/preprocessed_logs.csv"
anomaly_label_file = "/content/drive/MyDrive/DISTIL_FILES/input/anomaly_label.csv"
event_traces_output = "/content/drive/MyDrive/DISTIL_FILES/input/Event_traces.csv"

In [ ]:
# Load Preprocessed Logs
logs_df = pd.read_csv(preprocessed_logs_file)

In [ ]:
logs_df.head()

,BlockId,EventId,RawLog
0,blk_-1608999687919862906,E5,081109 203518 143 INFO dfs.DataNode$DataXceive...
1,blk_-1608999687919862906,E22,081109 203518 35 INFO dfs.FSNamesystem: BLOCK*...
2,blk_-1608999687919862906,E5,081109 203519 143 INFO dfs.DataNode$DataXceive...
3,blk_-1608999687919862906,E5,081109 203519 145 INFO dfs.DataNode$DataXceive...
4,blk_-1608999687919862906,E11,081109 203519 145 INFO dfs.DataNode$PacketResp...


In [ ]:
# Load Anomaly Labels
anomaly_labels_df = pd.read_csv(anomaly_label_file)
anomaly_labels_dict = dict(zip(anomaly_labels_df["BlockId"], anomaly_labels_df["Label"]))

**b. Grouping Logs by BlockId**

* The code groups all log lines by their BlockId.
* For each block, it creates an event sequence (a list of event IDs in the order they appeared).
* The anomaly label is looked up (defaulting to "Normal" if not found).
* These traces are saved as "event\_traces.csv".
* Outcome: Each block now has an associated sequence of events (its “trace”) and a ground-truth label.


In [ ]:
# Group logs by BlockId to create event sequences
event_traces = []
for block_id, group in logs_df.groupby("BlockId"):
    event_sequence = list(group["EventId"])  # Sequence of event templates

    # Get the anomaly label (default to "Normal" if missing)
    label = anomaly_labels_dict.get(block_id, "Normal")

    # Store event trace
    event_traces.append((block_id, label, event_sequence))

In [ ]:
# Convert to DataFrame and Save
event_traces_df = pd.DataFrame(event_traces, columns=["BlockId", "Label", "Features"])
event_traces_df.to_csv(event_traces_output, index=False)

print(f"Step 2 Complete: Event Traces saved to {event_traces_output}")

Step 2 Complete: Event Traces saved to /content/drive/MyDrive/DISTIL_FILES/input/Event_traces.csv


In [ ]:
event_traces_df[event_traces_df['Label'] == 'Anomaly']

,BlockId,Label,Features
33,blk_-1000723577943457888,Anomaly,"[E5, E22, E5, E7]"
53,blk_-1001299764911418845,Anomaly,"[E5, E5, E5, E22, E11, E9, E26, E11, E9, E11, ..."
304,blk_-1009207079038502874,Anomaly,"[E22, E5, E5, E5, E26, E26, E11, E9, E11, E9, ..."
305,blk_-1009284756580063377,Anomaly,"[E22, E5, E5, E5, E26, E26, E26, E11, E9, E11,..."
383,blk_-101140094574068250,Anomaly,"[E5, E22]"
...,...,...,...
574551,blk_985463838607094751,Anomaly,"[E5, E22]"
574690,blk_989409441141247289,Anomaly,"[E22, E5, E5, E5, E26, E26, E26, E11, E9, E11,..."
574790,blk_992101295951175683,Anomaly,"[E5, E5, E22, E5, E11, E9, E11, E9, E11, E9, E..."
574817,blk_992694869650988857,Anomaly,"[E5, E22]"


In [ ]:
event_traces_df[event_traces_df['Label'] == 'Normal']

,BlockId,Label,Features
0,blk_-1000002529962039464,Normal,"[E5, E5, E5, E22, E11, E9, E11, E9, E26, E26, ..."
1,blk_-100000266894974466,Normal,"[E22, E5, E5, E5, E26, E26, E26, E11, E9, E11,..."
2,blk_-1000007292892887521,Normal,"[E5, E5, E22, E5, E11, E9, E11, E9, E11, E9, E..."
3,blk_-1000014584150379967,Normal,"[E5, E22, E5, E5, E26, E26, E26, E11, E9, E11,..."
4,blk_-1000028658773048709,Normal,"[E5, E5, E5, E22, E11, E9, E11, E9, E11, E9, E..."
...,...,...,...
575056,blk_999905757185707736,Normal,"[E5, E5, E5, E22, E11, E9, E11, E9, E11, E9, E..."
575057,blk_999915040208161699,Normal,"[E5, E5, E5, E22, E11, E9, E11, E9, E26, E26, ..."
575058,blk_999958959261325562,Normal,"[E5, E22, E5, E5, E26, E11, E9, E11, E9, E11, ..."
575059,blk_999974850451006327,Normal,"[E5, E5, E5, E22, E11, E9, E11, E9, E11, E9, E..."


# **5. Splitting the Dataset for Model Training**

a. **Train–Validation–Test Split**

* The event traces are split into training (80%), validation (10%), and test (10%) sets.
* The split is stratified by the label to maintain the proportion of anomalies and normal cases in each set.
* Each split is saved to its own CSV file.

In [ ]:
# File Paths

train_output = "/content/drive/MyDrive/DISTIL_FILES/output/train_data.csv"
val_output = "/content/drive/MyDrive/DISTIL_FILES/output/val_data.csv"
test_output = "/content/drive/MyDrive/DISTIL_FILES/output/test_data.csv"
event_traces_output = "/content/drive/MyDrive/DISTIL_FILES/input/Event_traces.csv" # Define event_traces_output
# Import pandas
import pandas as pd  # Add this line to import pandas

# Import train_test_split
from sklearn.model_selection import train_test_split # Import the necessary function

# Load event traces
event_traces_df = pd.read_csv(event_traces_output)

# Split dataset into Training (80%), Temp (20%)
train_df, temp_df = train_test_split(event_traces_df, test_size=0.2, stratify=event_traces_df["Label"], random_state=42)

# Split Temp dataset into Validation (10%) and Test (10%)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df["Label"], random_state=42)

# Save the datasets
train_df.to_csv(train_output, index=False)
val_df.to_csv(val_output, index=False)
test_df.to_csv(test_output, index=False)

print(f"Data Splitting Complete:")
print(f"- Training Data: {train_output}")
print(f"- Validation Data: {val_output}")
print(f"- Test Data: {test_output}")

Data Splitting Complete:
- Training Data: /content/drive/MyDrive/DISTIL_FILES/output/train_data.csv
- Validation Data: /content/drive/MyDrive/DISTIL_FILES/output/val_data.csv
- Test Data: /content/drive/MyDrive/DISTIL_FILES/output/test_data.csv


# **6. Preparing Data for Transformer-based Classification**

**a. Data Loading and Preprocessing**

* Training and validation CSV files are loaded from Drive.
* The “Features” column (which is stored as a string representing a list) is converted back to an actual Python list using eval().
* The event sequence (list of event IDs) is flattened into a single string (each event separated by a space). This text will be used as the input for DistilBERT.
* The labels are encoded: “Normal” → 0 and “Anomaly” → 1.

In [ ]:
# File Paths

train_file = "/content/drive/MyDrive/DISTIL_FILES/output/train_data.csv"
val_file = "/content/drive/MyDrive/DISTIL_FILES/output/val_data.csv"

# Load Training & Validation Data
train_df = pd.read_csv(train_file)
val_df = pd.read_csv(val_file)

# Convert "Features" column (Event sequences) from string to list
train_df["Features"] = train_df["Features"].apply(eval)
val_df["Features"] = val_df["Features"].apply(eval)

# Flatten sequences into text format for tokenization
train_df["Text"] = train_df["Features"].apply(lambda x: " ".join(x))
val_df["Text"] = val_df["Features"].apply(lambda x: " ".join(x))


In [ ]:
# Encode Labels ("Normal" -> 0, "Anomaly" -> 1) using the same mapping for both datasets
label_mapping = {"Normal": 0, "Anomaly": 1}
train_df["Label"] = train_df["Label"].map(label_mapping)
val_df["Label"] = val_df["Label"].map(label_mapping)


**b. Tokenization**

* A pre-trained DistilBERT tokenizer is loaded.
* A function tokenize\_function is defined that:

  1. Tokenizes the text input, pads and truncates to a maximum length (64 tokens).
  2. Adds the corresponding label to the tokenized output.
* The Pandas DataFrames are converted to Hugging Face Dataset objects and tokenized.

In [ ]:
# Initialize DistilBERT Tokenizer
from transformers import DistilBertTokenizer # Importing DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
# tokenizer.add_special_tokens({"pad_token": "[PAD]"})  # Adding a padding token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
# Tokenization Function (Now Includes Labels)
def tokenize_function(examples):
    encoding = tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=64)
    encoding["labels"] = examples["Label"]  # Explicitly add labels
    return encoding

In [ ]:
!pip install datasets # Re-install the datasets module
from datasets import Dataset # Import the Dataset class from the datasets module

# Convert DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df[["Text", "Label"]])
val_dataset = Dataset.from_pandas(val_df[["Text", "Label"]])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
# Tokenize Datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/460048 [00:00<?, ? examples/s]

Map:   0%|          | 0/57506 [00:00<?, ? examples/s]

In [ ]:
# Remove extra columns and format labels correctly
train_dataset = train_dataset.remove_columns(["Text"])
val_dataset = val_dataset.remove_columns(["Text"])

# Set dataset format for PyTorch (Ensure `labels` are properly formatted)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

#**7. Model Setup, Training, and Saving**

**a. Loading and Configuring the Model**

* A DistilBERT model for sequence classification is loaded with 2 output labels.
* The token embeddings are resized to account for any new tokens in the tokenizer.

In [ ]:
# Load Pre-trained DistilBERT Model for Classification
from transformers import DistilBertForSequenceClassification # Importing the necessary class
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.resize_token_embeddings(len(tokenizer))  # Adjust for new tokens

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(30522, 768, padding_idx=0)

**b. Defining Training Arguments and Trainer**

**1. TrainingArguments:**

* Specifies output directories, evaluation strategy (per epoch), batch sizes, number of epochs (5), learning rate, weight decay, and saving strategy.
* Uses F1 score as the metric for saving the best model.

**2. Trainer:**

* Combines the model, training arguments, training and validation datasets.
* Outcome: The model is trained on the tokenized sequences of event IDs and saved.

In [ ]:
# Define Evaluation Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    accuracy = accuracy_score(labels, predictions)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
# # Training Arguments
# training_args = TrainingArguments(
#     output_dir="distilbert_log_model",
#     evaluation_strategy="epoch",  # Evaluate at each epoch
#     save_strategy="epoch",  # Save best model based on validation loss
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     learning_rate=5e-5,
#     weight_decay=0.01,
#     logging_dir="logs",
#     save_total_limit=2,  # Keep only last 2 models
#     load_best_model_at_end=True,
#     metric_for_best_model="loss",
#     greater_is_better=False,
# )

In [ ]:
!pip install --upgrade transformers  # Ensure transformers is up-to-date
from transformers import TrainingArguments # Import TrainingArguments

# Training Arguments
training_args = TrainingArguments(
    output_dir="distilbert_log_model",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",  # Save the model checkpoint at the end of each epoch
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="logs",
    save_total_limit=2,  # Keep only the last 2 checkpoints
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Use F1 score to select the best model
    greater_is_better=True,  # Higher F1 is better
    # report_to='none' # Turn off wandb if not using it.
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
# Trainer Setup (includes validation dataset)
from transformers import Trainer, DistilBertForSequenceClassification # Import the necessary classes

# Load the pre-trained DistilBERT model (make sure to specify num_labels)
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Adding validation dataset
    #compute_metrics=compute_metrics, # Add compute_metrics to Trainer if you've defined it
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'training_args' is not defined

In [ ]:
# Train Model & Save Best Version
trainer.train()

In [ ]:
# Save Model
model.save_pretrained("distilbert_log_model")
tokenizer.save_pretrained("distilbert_log_model_tokenizer")

print("Distilbert Model Training Complete! Model saved at /distilbert_log_model")

# **8. Anomaly Detection via Inference**

**a. Loading the Model and Test Data**

* The trained model and tokenizer are loaded.
* Test data is loaded and the “Features” column is converted from a string back to a list.

In [ ]:
# File Paths

model_path = "distilbert_log_model"
test_file = "/content/drive/MyDrive/DISTIL_FILES/output/train_data.csv"
output_report = "/content/drive/MyDrive/DISTIL_FILES/output/anomaly_label.csv"

# Load Trained Model & Tokenizer
tokenizer = DistilbertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)
model.eval()  # Set model to evaluation mode

# Load Test Data
test_df = pd.read_csv(test_file)

# Convert "Features" column (Event sequences) from string to list
test_df["Features"] = test_df["Features"].apply(eval)

**b. Defining a Function to Detect Anomalies**

* detect\_anomaly takes an event sequence.
* It assumes the sequence’s last event is the “actual” next event.
* The rest of the sequence is flattened into text and tokenized.
* The model is used to predict the next token by extracting logits for the last position.
* The function then retrieves the top‑K predicted tokens (using softmax and torch.topk).
* If the actual next event is not among the top‑K predictions, the function flags the block as anomalous.

In [ ]:
# Function to Check Anomalies
def detect_anomaly(event_sequence, top_k=5):
    input_text = " ".join(event_sequence[:-1])  # Input all events except last
    actual_next_event = event_sequence[-1]  # The actual next event

    # Tokenize Input
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate Predictions
    with torch.no_grad():
        output = model(input_ids)

    # Extract Last Logits
    logits = output.logits[:, -1, :]  # Get predictions for next token
    probs = torch.softmax(logits, dim=-1)  # Convert to probabilities

    # Get Top-K Predictions
    top_k_tokens = torch.topk(probs, top_k, dim=-1).indices[0].tolist()
    top_k_predictions = [tokenizer.decode([token]) for token in top_k_tokens]

    # Anomaly Detection
    if actual_next_event not in top_k_predictions:
        return {
            "Anomaly": True,
            "Reason": f"Expected '{actual_next_event}', but it was not in top-{top_k} predictions: {top_k_predictions}"
        }
    return {"Anomaly": False, "Reason": "Log sequence follows normal patterns"}

**c. Running Inference on the Test Set**

* The code loops over each test row, applies detect\_anomaly to the event sequence, and records whether an anomaly was detected along with the reason.
* The final anomaly report is saved as a CSV.
* Outcome: The report indicates which blocks’ event sequences deviate from what the model expects (i.e. anomalies).

In [ ]:
# Run Inference on Test Data
anomaly_results = []
for _, row in test_df.iterrows():
    block_id = row["BlockId"]
    label = row["Label"]
    features = row["Features"]

    # Detect Anomaly
    result = detect_anomaly(features, top_k=5)
    anomaly_results.append((block_id, label, result["Anomaly"], result["Reason"]))

# Save Anomaly Report
anomaly_report_df = pd.DataFrame(anomaly_results, columns=["BlockId", "Label", "Anomaly", "Reason"])
anomaly_report_df.to_csv(output_report, index=False)

print(f"Anomaly Detection Complete! Report saved to {output_report}")